# Скоринговая модель по поиску недооцененных акций на МосБирже.

В результате парсинга сайтов составляется топ недооценнных акций на основе мультипликаторов:



*   p/e - цена / прибыль; коэффициент прибыльности (<=10)
*   p/s - цена / объем продаж (<=2)
*   p/fcf - цена/движение свободных денежных средств (<20)
*   p/b - цена / балансовая стоимость (>= 0 & <=2)
*   ev/ebitda - за сколько лет окупится компания (>= 1 & <= 5)
*   divid - выплачивает ли дивиденды (>= 1)



In [ ]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import seaborn as sn
import matplotlib as plt

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive успешно подключен!")
else:
    print("❌ Google Drive не подключен")

Mounted at /content/drive
✅ Google Drive успешно подключен!


In [ ]:
ticker_list = [
               'ABIO', 'ABRD', 'AFKS', 'AFLT', 'AKRN', 'ALBK', 'ALRS', 'AMEZ', 'AQUA', 'ARSA', 'AVAN',
               'BANE', 'BISV', 'BLNG', 'BRZL', 'BSPB',
               'CBOM', 'CHKZ', 'CHMF', 'CHMK', 'CNRU', 'CNTL',
               'DIOD', 'DSKY', 'DVEC', 'DZRD',
               'EELT', 'ELFV', 'ELTZ', 'ENPG', 'ETLN',
               'FEES', 'FESH', 'FIXP', 'FLOT',
               'GAZA', 'GAZP', 'GCHE', 'GECO', 'GEMA', 'GEMC', 'GLTR', 'GMKN', 'GRNT', 'GTRK',
               'HIMC', 'HMSG', 'HYDR',
               'IGST', 'IRAO', 'IRKT',
               'JNOS',
               'KAZT', 'KBSB', 'KBTK', 'KCHE', 'KGKC', 'KLSB', 'KMAZ', 'KMEZ', 'KMTZ', 'KOGK', 'KRKN', 'KRKO', 'KROT', 'KRSB', 'KTSB', 'KUBE', 'KZOS',
               'LENT', 'LIFE', 'LKOH', 'LNZL', 'LPSB', 'LSNG', 'LSRG', 'LVHK',
               'MAGE', 'MAGN', 'MDMG', 'MFGS', 'MGNT', 'MGNZ', 'MGTS', 'MISB', 'MOEX', 'MRKC', 'MRKK', 'MRKP', 'MRKS', 'MRKU', 'MRKV', 'MRKY', 'MRKZ', 'MRSB', 'MSNG', 'MSRS', 'MSTT', 'MTLR', 'MTSS', 'MVID',
               'NAUK', 'NFAZ', 'NKHP', 'NKNC', 'NKSH', 'NLMK', 'NMTP', 'NNSB', 'NSVZ', 'NVTK',
               'OGKB', 'OKEY', 'ORUP', 'OZON',
               'PHOR', 'PIKK', 'PLZL', 'PMSB', 'POGR', 'POLY', 'POSI', 'PRFN',
               'QIWI',
               'RASP', 'RBCM', 'RENI', 'RGSS', 'RKKE', 'RNFT', 'ROLO', 'ROSB', 'ROST', 'RSTI', 'RTGZ', 'RTKM', 'RTSB', 'RUAL', 'RUGR', 'RZSB',
               'SAGO', 'SARE', 'SBER', 'SELG', 'SFIN', 'SGZH', 'SIBN', 'SLEN', 'SMLT', 'SNGS', 'SPBE', 'STSB', 'SVAV', 'SVET',
               'T', 'TASB', 'TATN', 'TGKA', 'TGKB', 'TGKD', 'TGKN', 'TNSE', 'TORS', 'TRMK', 'TRNFP', 'TTLK',
               'UCSS', 'UKUZ', 'UNAC', 'UNKL', 'UPRO', 'URKZ', 'USBN', 'UTAR', 'UWGN',
               'VGSB', 'VKCO', 'VLHZ', 'VRSB', 'VSMO', 'VSYDP', 'VTBR',
               'WTCMP', 'WUSH',
               'X5',
               'YAKG', 'YKEN', 'YRSB',
               'ZVEZ'
               ]


На smart-lab.ru найдем и и извлечем общие значения основных мультипликаторов для каждого тикера.

In [ ]:
# from autoscraper import AutoScraper
recomendated = []
result = []
df = pd.DataFrame()
index_cnt = 0


print(f'Парсим {len(ticker_list)} тикеров... Ждите 5 минут')

for ticker in ticker_list:
    remaining = len(ticker_list) - index_cnt - 1
    print(f'Парсим {ticker}{"." * (20 - len(ticker))} Осталось {remaining}')
    df.loc[index_cnt, 'ticker'] = ticker

    url = f'https://smart-lab.ru/forum/{ticker}'
    s = requests.Session()
    answer = s.post(url, headers=dict(Referer=url))  # work_plan = response 200
    soup = BeautifulSoup(answer.text, 'html.parser')  # .find_all('div', {'class': ['dhx_cal_data']})

    for t in soup.find_all('table', class_='fundam_table')[:-1]:
        words_list = t.text.split('\n')
        for w in words_list:
            if len(w) != 0:
                result.append(w)

    if len(result) <= 3:
        print(f'{ticker} данных нет')
        index_cnt += 1
        continue

    counter = 0
    key = ''

    for i in result:
        if '\t\t\t' in i:
            continue
        counter += 1
        if counter % 2 != 0:  # если остаток от деления не равен нулю, то это название колонки
            column_name = i
            continue
        else:
            df.loc[index_cnt, column_name] = i

    if ticker == 'SBER':
        print(result)

    result = []
    index_cnt += 1

print("парсинг окончен")
df.to_excel('/content/drive/MyDrive/Colab Notebooks/Скоринг акций/inform_table.xlsx')
print("файлик сохранен")

Парсим 189 тикеров... Ждите 5 минут
Парсим ABIO................ Осталось 188
Парсим ABRD................ Осталось 187
Парсим AFKS................ Осталось 186
Парсим AFLT................ Осталось 185
Парсим AKRN................ Осталось 184
Парсим ALBK................ Осталось 183
Парсим ALRS................ Осталось 182
Парсим AMEZ................ Осталось 181
Парсим AQUA................ Осталось 180
Парсим ARSA................ Осталось 179
Парсим AVAN................ Осталось 178
Парсим BANE................ Осталось 177
Парсим BISV................ Осталось 176
Парсим BLNG................ Осталось 175
Парсим BRZL................ Осталось 174
Парсим BSPB................ Осталось 173
Парсим CBOM................ Осталось 172
Парсим CHKZ................ Осталось 171
Парсим CHMF................ Осталось 170
Парсим CHMK................ Осталось 169
Парсим CNRU................ Осталось 168
Парсим CNTL................ Осталось 167
Парсим DIOD................ Осталось 166
Парсим DSKY..........

In [ ]:
df.iloc[14:18]

,ticker,Число акций ао,Номинал ао,Тикер ао,Капит-я,Выручка,EBITDA,Прибыль,Дивиденд ао,P/E,...,EV/EBITDA,Див.доход ао,Опер.доход,P/B,ЧПМ,Число акций ап,Номинал ап,Тикер ап,Дивиденд ап,Див.доход ап
14,BRZL,7 млн,0.025 руб,BRZL,"10,0 млрд",–,–,"1,4 млрд",–,"7,0",...,–,"0,0%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,BSPB,458 млн,1 руб,BSPB,"149,9 млрд",NaN,NaN,"46,1 млрд","46,33","3,2",...,NaN,"14,2%",–,"0,7","7,2%",20 млн,1 руб,BSPBP,"0,44","0,8%"
16,CBOM,33 430 млн,1 руб,CBOM,"253,9 млрд",NaN,NaN,"8,4 млрд",–,"30,4",...,NaN,"0,0%","55,7 млрд","0,7","2,0%",NaN,NaN,NaN,NaN,NaN
17,CHKZ,1 млн,200 руб,CHKZ,"10,0 млрд","98,0 млрд","17,4 млрд","-0,3 млрд",720,"-32,9",...,"2,0","4,3%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Извлечем значения fcf для каждого тикера

In [ ]:
fcf_df = pd.DataFrame()


def get_fcf():
    url = f'https://smart-lab.ru/q/shares_fundamental/?field=fcf_share'
    s = requests.Session()
    answer = s.post(url, headers=dict(Referer=url))
    soup = BeautifulSoup(answer.text, 'html.parser')

    temp = soup.find('table', class_='simple-little-table trades-table')

    words_list = temp.text


    counter = 0

    meaning_flag = None
    ticker_flag = None

    actual_new_ticker_list = []
    meanings = []

    permited_word_exceptions_list = ['X5', 'T']

    garbigge_list = ['ВинЛаб']

    for i in words_list.split('\n'):
        i = i.replace('\t', '')
        if len(i) < 3:
            continue

        if i in garbigge_list:
            continue

        if i[1].lower() in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
            ticker_flag = False # это какой-то иной русский текст - не тикер
            print(f'\n{i} - определили что это не тикер')
            continue

        # 1) ищем сначала что это тикер
        if (len(i) == 4 and '.' not in i) or (len(i) == 5 and i[:-1] == 'P') or i in permited_word_exceptions_list:
            counter += 1

            if i.isalpha():
                if i.capitalize() in actual_new_ticker_list:
                    ticker_flag = False
                    continue

                if i not in actual_new_ticker_list:  # Только уникальные
                    actual_new_ticker_list.append(i)
                    print(f'{counter}) В список actual_new_ticker_list добавлено {i}')

                    ticker_flag = True # устанавливается True в случае если только что записали тикер (под i был тикер)
                    meaning_flag = False # устанавливается False если это было не значение

                    continue

        # 2) Ищем число - значение тикера
        if any(char.isdigit() for char in i) and (not i.isalpha()):
            if ticker_flag:  # Только если предыдущий был тикер
                print(f'{counter}) В список meanings добавлено {i}')
                meanings.append(i)
                meaning_flag = True
                ticker_flag = False

    return actual_new_ticker_list, meanings


actual_new_ticker_list, meanings = get_fcf()

# print('sorted', sorted(actual_new_ticker_list))

print(len(actual_new_ticker_list), actual_new_ticker_list)
print(len(meanings), meanings)

fcf_df['ticker'] = actual_new_ticker_list
fcf_df['fcf'] = meanings
fcf_df.tail()


Название - определили что это не тикер

Тикер - определили что это не тикер

 Изм. %, г/г - определили что это не тикер

Энергия РКК - определили что это не тикер
1) В список actual_new_ticker_list добавлено RKKE
1) В список meanings добавлено 3 878

Павловский автобус - определили что это не тикер
2) В список actual_new_ticker_list добавлено PAZA
2) В список meanings добавлено 2 852

ЧМК - определили что это не тикер
3) В список actual_new_ticker_list добавлено CHMK
3) В список meanings добавлено 2 670

Лукойл - определили что это не тикер
4) В список actual_new_ticker_list добавлено LKOH
4) В список meanings добавлено 1 413

Ижсталь - определили что это не тикер
5) В список actual_new_ticker_list добавлено IGST
5) В список meanings добавлено 1 301

ТНС энерго - определили что это не тикер
6) В список actual_new_ticker_list добавлено TNSE
6) В список meanings добавлено 881.6
7) В список actual_new_ticker_list добавлено HEAD
7) В список meanings добавлено 513.5

ВСМПО-АВИСМА - определ

,ticker,fcf
180,NFAZ,-421.0
181,AKRN,-522.3
182,LSRG,-538.7
183,SMLT,-4 727
184,CHKZ,-13 975


Приджоинить названия компаний

In [ ]:
cf = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Скоринг акций/company dict.xlsx')
cf.head(3)

,ticker,name
0,SFTL,Софтлайн
1,VSYDP,Выборгский судостроительный завод
2,ISKJ,Артген биотех (ИСКЧ)


In [ ]:
df[df['ticker'] == 'SBER']

,ticker,Число акций ао,Номинал ао,Тикер ао,Капит-я,Выручка,EBITDA,Прибыль,Дивиденд ао,P/E,...,EV/EBITDA,Див.доход ао,Опер.доход,P/B,ЧПМ,Число акций ап,Номинал ап,Тикер ап,Дивиденд ап,Див.доход ап
142,SBER,21 587 млн,3 руб,SBER,"6 901,9 млрд",NaN,NaN,"1 661,3 млрд","34,84","4,2",...,NaN,"11,4%","4 510,3 млрд","0,9","6,1%",1 000 млн,3 руб,SBERP,"34,84","11,5%"


In [ ]:
mf = pd.merge(df, fcf_df, on='ticker', how='left')
mf = pd.merge(mf, cf, on='ticker', how='left')
mf.tail(3)

,ticker,Число акций ао,Номинал ао,Тикер ао,Капит-я,Выручка,EBITDA,Прибыль,Дивиденд ао,P/E,...,Опер.доход,P/B,ЧПМ,Число акций ап,Номинал ап,Тикер ап,Дивиденд ап,Див.доход ап,fcf,name
186,YKEN,10 188 млн,2.22 руб,YKEN,"4,9 млрд","39,7 млрд","-14,1 млрд","-28,3 млрд",–,"-0,2",...,NaN,NaN,NaN,1 333 млн,2.22 руб,YKENP,–,"0,0%",-2.16,Якутскэнерго
187,YRSB,17 млн,1 руб,YRSB,"14,8 млрд","21,2 млрд","1,1 млрд","0,7 млрд",–,"21,9",...,NaN,NaN,NaN,5 млн,1 руб,YRSBP,–,"0,0%",27.6,ТНС энерго Ярославль
188,ZVEZ,562 млн,0.1 руб,ZVEZ,"3,9 млрд","3,5 млрд",–,"-0,2 млрд",–,"-25,3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.30,Звезда


In [ ]:
df = mf

## Предобработка данных

переименуем столбцы

In [ ]:
df[df['ticker'] == 'SBER']

,ticker,Число акций ао,Номинал ао,Тикер ао,Капит-я,Выручка,EBITDA,Прибыль,Дивиденд ао,P/E,...,Опер.доход,P/B,ЧПМ,Число акций ап,Номинал ап,Тикер ап,Дивиденд ап,Див.доход ап,fcf,name
142,SBER,21 587 млн,3 руб,SBER,"6 901,9 млрд",NaN,NaN,"1 661,3 млрд","34,84","4,2",...,"4 510,3 млрд","0,9","6,1%",1 000 млн,3 руб,SBERP,"34,84","11,5%",NaN,Сбербанк


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ticker          189 non-null    object
 1   Число акций ао  188 non-null    object
 2   Номинал ао      188 non-null    object
 3   Тикер ао        172 non-null    object
 4   Капит-я         188 non-null    object
 5   Выручка         179 non-null    object
 6   EBITDA          179 non-null    object
 7   Прибыль         188 non-null    object
 8   Дивиденд ао     172 non-null    object
 9   P/E             180 non-null    object
 10  P/S             173 non-null    object
 11  P/BV            173 non-null    object
 12  EV/EBITDA       173 non-null    object
 13  Див.доход ао    168 non-null    object
 14  Опер.доход      9 non-null      object
 15  P/B             7 non-null      object
 16  ЧПМ             7 non-null      object
 17  Число акций ап  49 non-null     object
 18  Номинал ап

In [ ]:
df = df.set_axis(['ticker',
                  'stock_cnt',
                  'nominal',
                  'ticker_ao',
                  'capitalization',
                  'revenue', # выручка
                  'ebitda',
                  'profit', # прибыль
                  'divid',
                  'p_e',
                  'p_s',
                  'p_bv',
                  'ev_ebitda',
                  'div_income_ao',
                  'oper_income',
                  'p_b',
                  'margin',
                  'cnt_stock_ap',
                  'nominal_ap',
                  'ticker_ap',
                  'dividend_ap',
                  'div_incom_ap',
                  'p_fcf',
                  'name'], # 24
                 axis='columns')
df = df[['name', 'ticker', 'p_e', 'p_s', 'p_bv',  'p_fcf', 'ev_ebitda', 'stock_cnt', 'nominal', 'capitalization', 'revenue', 'ebitda', 'profit', 'divid', ]] # 14
df.head()

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
0,Артген биотех (ИСКЧ),ABIO,"25,8","3,9","-19,4",2.00,"5,8",93 млн,0.1 руб,"6,4 млрд","1,6 млрд","1,1 млрд","0,2 млрд",–
1,Абрау-Дюрсо,ABRD,"12,2","1,1","1,2",-26.0,"8,4",98 млн,1 руб,"15,6 млрд","14,1 млрд","2,9 млрд","1,3 млрд","5,01"
2,АФК Система,AFKS,"-0,8","0,1",–,-22.1,"0,4",9 650 млн,0.09 руб,"131,7 млрд","1 315,9 млрд","362,3 млрд","-155,4 млрд",–
3,Аэрофлот,AFLT,"2,3","0,3","5,8",4.81,"2,7",3 976 млн,1 руб,"225,6 млрд","896,7 млрд","277,4 млрд","97,1 млрд","5,27"
4,Акрон,AKRN,"13,9","2,8","3,0",-522.3,"8,8",37 млн,5 руб,"653,7 млрд","232,9 млрд","87,6 млрд","46,9 млрд",723


In [ ]:
df['stock_cnt'] = df['stock_cnt'].apply(lambda x: str(x).replace(' млн', "000000"))
df["revenue"] = df["revenue"].apply(lambda x: str(x).replace(' млрд', "00000000")) # Идет пять нолей потому что перед словом млрд еще одна десятая цифра
df['capitalization'] = df["capitalization"].apply(lambda x: str(x).replace(' млрд', "00000000"))
df['ebitda'] = df["ebitda"].apply(lambda x: str(x).replace(' млрд', "00000000"))
df['profit'] = df["profit"].apply(lambda x: str(x).replace(' млрд', "00000000"))
df['nominal'] = df["nominal"].apply(lambda x: str(x).replace(' руб', ""))
df.head()

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
0,Артген биотех (ИСКЧ),ABIO,"25,8","3,9","-19,4",2.00,"5,8",93000000,0.1,"6,400000000","1,600000000","1,100000000","0,200000000",–
1,Абрау-Дюрсо,ABRD,"12,2","1,1","1,2",-26.0,"8,4",98000000,1,"15,600000000","14,100000000","2,900000000","1,300000000","5,01"
2,АФК Система,AFKS,"-0,8","0,1",–,-22.1,"0,4",9 650000000,0.09,"131,700000000","1 315,900000000","362,300000000","-155,400000000",–
3,Аэрофлот,AFLT,"2,3","0,3","5,8",4.81,"2,7",3 976000000,1,"225,600000000","896,700000000","277,400000000","97,100000000","5,27"
4,Акрон,AKRN,"13,9","2,8","3,0",-522.3,"8,8",37000000,5,"653,700000000","232,900000000","87,600000000","46,900000000",723


убираем запятые


In [ ]:
# Словарь с правилами для каждой колонки
cleaning_rules = {
    'revenue': {',': '', ' ': '', '–': '0', 'nan': '0'},
    'capitalization': {',': '', ' ': '', '-': '0', 'nan': '0'},
    'ebitda': {',': '', ' ': '', '–': '0', 'nan': '0'},
    'stock_cnt': {' ': '', '–': '0', 'nan': '0'},
    'p_e': {',': '.', '–': '0.0'},
    'p_s': {',': '.', '–': '0.0'},
    'p_bv': {',': '.', '–': '0.0'},
    'ev_ebitda': {',': '.', '–': '-1.0'},
    'p_fcf': {' ': '', ',': '.'},
    'divid': {'–': '0', ',': '.'},
    'profit': {',': '', ' ': '', '–': '0', 'nan': '0'},
}

# Применяем правила
for col, rules in cleaning_rules.items():
    df[col] = df[col].astype(str)
    for old, new in rules.items():
        df[col] = df[col].str.replace(old, new, regex=False)

df.tail()

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
184,NaN,X5,9.5,0.2,-80.2,nan,4.0,272000000,1164.26,769800000000,4481700000000,258200000000,81200000000,1016
185,ЯТЭК,YAKG,40.7,3.8,1.9,-0.75,11.8,827000000,1,35000000000,9300000000,3500000000,0900000000,0
186,Якутскэнерго,YKEN,-0.2,0.1,-0.2,-2.16,-5.4,10188000000,2.22,4900000000,39700000000,-14100000000,-28300000000,0
187,ТНС энерго Ярославль,YRSB,21.9,0.7,13.7,27.6,13.4,17000000,1,14800000000,21200000000,1100000000,0700000000,0
188,Звезда,ZVEZ,-25.3,1.1,-1.9,0.30,-1.0,562000000,0.1,3900000000,3500000000,0,-0200000000,0


In [ ]:
df = df.fillna(0)
df

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
0,Артген биотех (ИСКЧ),ABIO,25.8,3.9,-19.4,2.00,5.8,93000000,0.1,6400000000,1600000000,1100000000,0200000000,0
1,Абрау-Дюрсо,ABRD,12.2,1.1,1.2,-26.0,8.4,98000000,1,15600000000,14100000000,2900000000,1300000000,5.01
2,АФК Система,AFKS,-0.8,0.1,0.0,-22.1,0.4,9650000000,0.09,131700000000,1315900000000,362300000000,-155400000000,0
3,Аэрофлот,AFLT,2.3,0.3,5.8,4.81,2.7,3976000000,1,225600000000,896700000000,277400000000,97100000000,5.27
4,Акрон,AKRN,13.9,2.8,3.0,-522.3,8.8,37000000,5,653700000000,232900000000,87600000000,46900000000,723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0,X5,9.5,0.2,-80.2,nan,4.0,272000000,1164.26,769800000000,4481700000000,258200000000,81200000000,1016
185,ЯТЭК,YAKG,40.7,3.8,1.9,-0.75,11.8,827000000,1,35000000000,9300000000,3500000000,0900000000,0
186,Якутскэнерго,YKEN,-0.2,0.1,-0.2,-2.16,-5.4,10188000000,2.22,4900000000,39700000000,-14100000000,-28300000000,0
187,ТНС энерго Ярославль,YRSB,21.9,0.7,13.7,27.6,13.4,17000000,1,14800000000,21200000000,1100000000,0700000000,0


In [ ]:
df[df['ticker'] == 'SBER']

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
142,Сбербанк,SBER,4.2,nan,nan,nan,nan,21587000000,3,6901900000000,0,0,1661300000000,34.84


In [ ]:
df['p_e'] = df['p_e'].str.replace(' ', '')

In [ ]:
df[['stock_cnt', 'capitalization', 'revenue', 'ebitda', 'profit']] = df[['stock_cnt', 'capitalization', 'revenue', 'ebitda', 'profit']].astype('int64')

columns = ['p_e', 'p_s', 'p_bv', 'ev_ebitda', 'p_fcf', 'divid']

# Убираем пробелы, знак % и конвертируем
df[columns] = df[columns].astype(str).apply(
    lambda x: x.str.replace(' ', '').str.replace('%', '')
).astype('float64')

df.head(2)

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
0,Артген биотех (ИСКЧ),ABIO,25.8,3.9,-19.4,2.0,5.8,93000000,0.1,6400000000,1600000000,1100000000,200000000,0.00
1,Абрау-Дюрсо,ABRD,12.2,1.1,1.2,-26.0,8.4,98000000,1,15600000000,14100000000,2900000000,1300000000,5.01


In [ ]:
df[df['ticker'] == 'SBER']

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid
142,Сбербанк,SBER,4.2,NaN,NaN,NaN,NaN,21587000000,3,6901900000000,0,0,1661300000000,34.84


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            189 non-null    object 
 1   ticker          189 non-null    object 
 2   p_e             180 non-null    float64
 3   p_s             173 non-null    float64
 4   p_bv            173 non-null    float64
 5   p_fcf           148 non-null    float64
 6   ev_ebitda       173 non-null    float64
 7   stock_cnt       189 non-null    int64  
 8   nominal         189 non-null    object 
 9   capitalization  189 non-null    int64  
 10  revenue         189 non-null    int64  
 11  ebitda          189 non-null    int64  
 12  profit          189 non-null    int64  
 13  divid           172 non-null    float64
dtypes: float64(6), int64(5), object(3)
memory usage: 20.8+ KB


In [ ]:
df[df.name.isnull()]

,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid


In [ ]:
df['price'] = round(df.capitalization / df.stock_cnt, 1)
df.head(3)



,name,ticker,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid,price
0,Артген биотех (ИСКЧ),ABIO,25.8,3.9,-19.4,2.0,5.8,93000000,0.1,6400000000,1600000000,1100000000,200000000,0.00,68.8
1,Абрау-Дюрсо,ABRD,12.2,1.1,1.2,-26.0,8.4,98000000,1,15600000000,14100000000,2900000000,1300000000,5.01,159.2
2,АФК Система,AFKS,-0.8,0.1,0.0,-22.1,0.4,9650000000,0.09,131700000000,1315900000000,362300000000,-155400000000,0.00,13.6


In [ ]:
df = df[[
    'name',
    'ticker',
    'price',
    'p_e',
    'p_s',
    'p_bv',
    'p_fcf',
    'ev_ebitda',
    'stock_cnt',
    'nominal',
    'capitalization',
    'revenue',
    'ebitda',
    'profit',
    'divid'
    ]]


In [ ]:
df['score'] = 0
df.head(2)

/tmp/ipython-input-1903093238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = 0


,name,ticker,price,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,ebitda,profit,divid,score
0,Артген биотех (ИСКЧ),ABIO,68.8,25.8,3.9,-19.4,2.0,5.8,93000000,0.1,6400000000,1600000000,1100000000,200000000,0.00,0
1,Абрау-Дюрсо,ABRD,159.2,12.2,1.1,1.2,-26.0,8.4,98000000,1,15600000000,14100000000,2900000000,1300000000,5.01,0


In [ ]:
df.columns

Index(['name', 'ticker', 'price', 'p_e', 'p_s', 'p_bv', 'p_fcf', 'ev_ebitda',
       'stock_cnt', 'nominal', 'capitalization', 'revenue', 'ebitda', 'profit',
       'divid', 'score'],
      dtype='object')

In [ ]:
df = df[['name', 'score', 'ticker',  'price', 'p_e', 'p_s', 'p_bv', 'p_fcf', 'ev_ebitda', 'stock_cnt',
       'nominal', 'capitalization', 'revenue', 'divid', 'profit']] #
df.head(2)

,name,score,ticker,price,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,divid,profit
0,Артген биотех (ИСКЧ),0,ABIO,68.8,25.8,3.9,-19.4,2.0,5.8,93000000,0.1,6400000000,1600000000,0.00,200000000
1,Абрау-Дюрсо,0,ABRD,159.2,12.2,1.1,1.2,-26.0,8.4,98000000,1,15600000000,14100000000,5.01,1300000000


In [ ]:
print(df.columns.tolist())

['name', 'score', 'ticker', 'price', 'p_e', 'p_s', 'p_bv', 'p_fcf', 'ev_ebitda', 'stock_cnt', 'nominal', 'capitalization', 'revenue', 'divid', 'profit']


In [ ]:
df[df['p_e'].isna()]

,name,score,ticker,price,p_e,p_s,p_bv,p_fcf,ev_ebitda,stock_cnt,nominal,capitalization,revenue,divid,profit
5,Бест Эффортс Банк,0,ALBK,64.3,NaN,NaN,NaN,NaN,NaN,56000000,—,3600000000,0,NaN,200000000
12,Башинформсвязь,0,BISV,0.3,NaN,NaN,NaN,2.90,NaN,949000000,—,300000000,26500000000,0.0,800000000
33,Fix Price,0,FIXP,NaN,NaN,NaN,NaN,NaN,NaN,0,nan,0,0,NaN,0
77,Магаданэнерго,0,MAGE,3.7,NaN,NaN,NaN,-5.02,NaN,464000000,1,1700000000,16300000000,0.0,-3600000000
119,PETROPAVLOVSK,0,POGR,24.7,NaN,NaN,NaN,NaN,NaN,3957000000,—,97600000000,61200000000,NaN,-1500000000
131,Росбанк,0,ROSB,0.0,NaN,NaN,NaN,NaN,NaN,1551000000,—,0,0,NaN,29100000000
133,Россети,0,RSTI,1.3,NaN,NaN,NaN,NaN,NaN,195996000000,—,255300000000,1081000000000,NaN,58600000000
171,Уральская кузница,0,URKZ,256.4,NaN,NaN,NaN,-84.00,NaN,55000000,0.01,14100000000,22500000000,0.0,9000000000
180,Выборгский судостроительный завод,0,VSYDP,7750.0,NaN,NaN,NaN,NaN,NaN,2000000,0.5,15500000000,12100000000,0.0,-200000000


# Определим ТОП акций с наивысшим рейтингом

In [ ]:
df['score'] = 0

df['fcf_fl'] = np.where((df['p_fcf'] > 1) & (df['p_fcf'] < 20), 1, 0)
df['pe_fl'] = np.where((df['p_e'] >= 7) & (df['p_e'] <= 10), 1, 0)
df['ps_fl'] = np.where(df['p_s'] <= 2, 1, 0)
df['ev_fl'] = np.where((df['ev_ebitda'] >= 1) & (df['ev_ebitda'] <= 5), 1, 0)
df['pb_fl'] = np.where((df['p_bv'] >= 0) & (df['p_bv'] <= 2), 1.5, 0)
df['div_fl'] = np.where(df['divid'] >= 1, 1.5, 0)

cols = ['fcf_fl', 'pe_fl', 'ps_fl', 'ev_fl', 'pb_fl', 'div_fl']

df['score'] = df[cols].sum(axis=1)
df = df.sort_values('score', ascending = False)

df = df[['name', 'score', 'ticker', 'price', 'p_e', 'pe_fl', 'p_s', 'ps_fl', 'p_bv', 'pb_fl', 'p_fcf', 'fcf_fl',
       'ev_ebitda', 'ev_fl', 'divid', 'div_fl', 'stock_cnt', 'revenue']] # , 'profit'

print("Топ самых недооцененных акций")
df.head(10)

Топ самых недооцененных акций


,name,score,ticker,price,p_e,pe_fl,p_s,ps_fl,p_bv,pb_fl,p_fcf,fcf_fl,ev_ebitda,ev_fl,divid,div_fl,stock_cnt,revenue
56,Курганская ГК,6.0,KGKC,46.7,-57.0,0,0.5,1,0.7,1.5,2.73,1,4.9,1,1.302300,1.5,122000000,10700000000
71,Лукойл,6.0,LKOH,5534.9,7.0,1,0.5,1,0.7,1.5,1413.00,0,2.6,1,938.000000,1.5,693000000,7890000000000
146,Газпромнефть,6.0,SIBN,487.9,7.1,1,0.6,1,0.8,1.5,67.20,0,3.0,1,44.510000,1.5,4741000000,3765000000000
74,Россети Ленэнерго,5.5,LSNG,18.6,7.0,1,1.2,1,0.7,1.5,2.56,1,2.2,1,0.428100,0.0,8524000000,131100000000
11,Башнефть,5.0,BANE,1640.5,2.3,0,0.2,1,0.3,1.5,NaN,0,1.3,1,147.310000,1.5,148000000,1143000000000
37,Черкизово,5.0,GCHE,3481.0,10.7,0,0.5,1,0.0,1.5,-174.60,0,2.9,1,98.920000,1.5,42000000,281400000000
52,КуйбышевАзот,5.0,KAZT,433.9,8.2,1,0.9,1,0.8,1.5,-0.57,0,5.1,0,13.500000,1.5,177000000,81500000000
53,ТНС энерго Кубань,5.0,KBSB,794.4,4.4,0,0.2,1,1.5,1.5,137.50,0,3.7,1,50.365000,1.5,18000000,79400000000
17,ЧКПЗ,5.0,CHKZ,10000.0,-32.9,0,0.1,1,1.1,1.5,-13975.00,0,2.0,1,720.000000,1.5,1000000,98000000000
84,ТНС энерго Марий Эл,5.0,MISB,63.1,6.6,0,0.8,1,1.8,1.5,6.58,1,7.2,0,3.283019,1.5,122000000,9900000000


In [ ]:
df[df['ticker'] == 'SBER']

,name,score,ticker,price,p_e,pe_fl,p_s,ps_fl,p_bv,pb_fl,p_fcf,fcf_fl,ev_ebitda,ev_fl,divid,div_fl,stock_cnt,revenue
142,Сбербанк,1,SBER,319.7,4.2,0,NaN,0,NaN,0,NaN,0,NaN,0,34.84,1,21587000000,0


### Посмотреть конкретных эмитентов

In [ ]:
df.query('ticker == "SBER"')

,name,score,ticker,price,p_e,pe_fl,p_s,ps_fl,p_bv,pb_fl,p_fcf,fcf_fl,ev_ebitda,ev_fl,divid,div_fl,stock_cnt,revenue
142,Сбербанк,1,SBER,319.7,4.2,0,NaN,0,NaN,0,NaN,0,NaN,0,34.84,1,21587000000,0


In [ ]:
df[df['name'].str.contains('Мечел', na=False)]

,name,score,ticker,price,p_e,pe_fl,p_s,ps_fl,p_bv,pb_fl,p_fcf,fcf_fl,ev_ebitda,ev_fl,divid,div_fl,stock_cnt,revenue
98,Мечел,1,MTLR,96.4,-0.7,0,0.1,1,-0.2,0,-8.94,0,9.3,0,0.0,0,416000000,333800000000
